In [1]:
import pandas as pd
import altair as alt

In [2]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [6]:
df = pd.read_csv('wdi.csv')
df['gdp_capita'] = df.gdp / df.population
data = df[df.year==2020]
data.head()

,iso3,iso2,name,capital,continent,region,income_level,year,population,gdp,...,hospital_beds_1000,physicians_1000,maternal_death_risk,suicides_100k,health_expenditure_share,infant_mortality_1000,renewables_share,co2_capita,greenhousegas_capita,gdp_capita
13,CHL,CL,Chile,Santiago,Americas,Latin America & Caribbean,High income,2020,19300315.0,4.813737e+11,...,NaN,2.8351,0.026083,NaN,9.753005,5.9,NaN,4.395151,106722.334200,24941.235088
32,GIB,GI,Gibraltar,NaN,Europe,Europe & Central Asia,High income,2020,32709.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,MNP,MP,Northern Mariana Islands,Saipan,Oceania,East Asia & Pacific,High income,2020,49587.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,FSM,FM,"Micronesia, Fed. Sts.",Palikir,Oceania,East Asia & Pacific,Lower middle income,2020,112106.0,4.091137e+08,...,NaN,0.9391,0.204893,NaN,11.559997,21.6,NaN,0.958914,194.111345,3649.347298
99,CIV,CI,Cote d'Ivoire,Yamoussoukro,Africa,Sub-Saharan Africa,Lower middle income,2020,26811790.0,1.441776e+11,...,NaN,NaN,2.185712,NaN,3.722458,57.5,NaN,0.406347,26412.281150,5377.394446


# 1. Interaction Types

## 1.1 Point selection

- **Use case:** Highlighting individual points / bars / lines via click, mouseover inside Plot

- **Logic:**
    - 1. Define selection logic: `selection_point()`
    - 2. Link the selection as a Parameter to the Altair Chair: `add_params()`
    - 3. Implement Highlighting via conditional coloring/sizing/opacity: `condition()`
- **Usage**: Shift + click/mouseover/etc. for selection of multiple points


In [28]:
alt.Chart(data).mark_circle().encode(
    x = 'gdp_capita',
    y = alt.Y('life_expectancy').scale(zero=False),
    color = alt.condition(alt.datum.life_expectancy>0, if_true='region', if_false=alt.value('#385b94')),
).properties(width='container')

alt.Chart(...)

In [46]:
selection = alt.selection_point(fields=['name'], on='click', empty=False, nearest=True)

points = alt.Chart(data).mark_circle().encode(
    x = 'gdp_capita',
    y = alt.Y('life_expectancy').scale(zero=False),
    color = alt.condition(predicate=selection, if_true=alt.value('red'), if_false=alt.value('#385b94')),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.3)),
    size = alt.condition(selection, alt.value(300), alt.value(50)),
    tooltip = ['name','life_expectancy','gdp_capita']
).properties(width='container').add_params(selection)

text = alt.Chart(data).mark_text().encode(
    x = 'gdp_capita',
    y = alt.Y('life_expectancy').scale(zero=False),
    text = alt.condition(predicate=selection, if_true='name', if_false=alt.value(''))
).properties(width='container').add_params(selection)

points + text

alt.LayerChart(...)

In [50]:
selection = alt.selection_point(fields=['region'], on='click', empty=False, nearest=True)

points = alt.Chart(data).mark_circle().encode(
    x = 'gdp_capita',
    y = alt.Y('life_expectancy').scale(zero=False),
    color = 'region',
    opacity=alt.condition(selection, alt.value(1), alt.value(0.3)),
    size = alt.condition(selection, alt.value(300), alt.value(50)),
    tooltip = ['name','region','life_expectancy','gdp_capita']
).properties(width='container').add_params(selection)

points

alt.Chart(...)

In [16]:
hover = alt.selection_point(

    # Select according to the following data fields
    fields=["name"], 

    # Nearest point or exact point
    nearest=True, 

    # Events to listen to: "mouseover", "click", "wheel", "mouseup", ...
    on="click", 

    # True: Multi-select, False: single select
    toggle=False,

    # How should an empty selection behave? True, False
    empty=False
)


alt.Chart(data).mark_circle().encode(
    x='gdp_capita:Q',
    y=alt.Y('life_expectancy:Q').scale(zero=False),
    color='region:N',
    tooltip=['name:N', 'continent:N', 'life_expectancy:Q', 'gdp_capita:Q'],
    opacity=alt.condition(hover, alt.value(1), alt.value(0.3)),
    size = alt.condition(hover, alt.value(100), alt.value(30))
).add_params(hover).properties(width='container')


alt.Chart(...)

In [ ]:
click = alt.selection_point(empty=False, on='mouseover')
alt.Chart(data).mark_bar(tooltip=True).encode(
    x='count()',
    y='region',
    color = alt.condition(click, alt.value('red'), alt.value('lightgrey')),
).add_params(click).properties(width='container')

## 1.2 Interval selection

In [60]:
selection = alt.selection_interval(
    encodings=['x','y'],
    translate=True,
    zoom=True)

alt.Chart(data).mark_circle(tooltip=True, size = 50).encode(
    x=alt.X('gdp_capita:Q').scale(zero=False),
    y=alt.Y('life_expectancy:Q').scale(zero=False),
    color=alt.condition(selection, 'region', alt.value('lightgrey'))
).add_params(selection).properties(width='container')

alt.Chart(...)

## 1.3 Selection via Legend

In [56]:
legend_click = alt.selection_point(
    fields=['region'], 
    bind='legend', 
    empty=False,
    toggle=True)

alt.Chart(data).mark_circle().encode(
    x='gdp_capita:Q',
    y=alt.Y('life_expectancy:Q').scale(zero=False),
    color='region:N',
    opacity=alt.condition(legend_click, alt.value(1), alt.value(0.2)),
    size = alt.condition(legend_click, alt.value(300), alt.value(30))
).add_params(legend_click).properties(width='container')

alt.Chart(...)

## 1.4 Selection via Input Widgets

- Radio Buttons (`binding_radio`)
- Select Dropdown (`binding_select`)
- Checkbox (`binding_checkbox`)
- Slider (`binding_range`)

In [64]:
regions = data.region.unique()
input_dropdown = alt.binding_select(options=regions, name='Please select a Region')
#input_radio = alt.binding_radio(options=regions, name='Region')

widget_selection = alt.selection_point(
    fields=['region'], 
    bind=input_dropdown, 
    name='Select', 
    empty=False, 
    value='South Asia')

alt.Chart(data).mark_circle().encode(
    x='gdp_capita:Q',
    y=alt.Y('life_expectancy:Q').scale(zero=False),
    tooltip=['name:N'],
    color=alt.condition(widget_selection, 'region:N', alt.value('lightgray'),legend=None)
).add_params(widget_selection).properties(width='container')

alt.Chart(...)

# 2. Linking Multiple Plots

## 2.1 Roll-up
**Use case:** Going from detailed view to aggregated view

In [77]:
alt.Chart(data[data.gdp_capita>40000]).mark_bar().encode(
    y='region:N',
    x='count():Q'
)

alt.Chart(...)

In [85]:
brush = alt.selection_interval(

    # Enable or disable panning
    translate=True,
    # Enable or disable zooming
    zoom=True
)

points = alt.Chart(data).mark_point().encode(
    x='gdp_capita:Q',
    y=alt.Y('life_expectancy:Q').scale(zero=False),
    color=alt.condition(brush, 'region:N', alt.value('lightgray'))
).add_params(brush)

bar = alt.Chart(data).mark_bar().encode(
    y='region:N',
    color = 'region:N',
    x='count():Q'
).transform_filter(brush)

le = alt.Chart(data).mark_bar().encode(
    y='region:N',
    color = 'region:N',
    x='mean(life_expectancy):Q'
).transform_filter(brush)

alt.hconcat(
    points,
    alt.vconcat(bar, le)
)

alt.HConcatChart(...)

## 2.2 Drill Down

**Use case:** Going from aggregated view to detailed view

In [98]:
click = alt.selection_point(fields=['region'], on='click', toggle=True)

bars = alt.Chart(data).mark_bar().encode(
    y='region:N',
    x='count():Q',
    color=alt.condition(click, 'region:N', alt.value('lightgray'))
).add_params(click).properties(width = 200, height=300)

points = alt.Chart(data).mark_point().encode(
    x=alt.Y('gdp_capita:Q').scale(domain=[0,120000]),
    y=alt.Y('life_expectancy:Q').scale(domain=[50,90]),
    color=alt.condition(click, 'region:N', alt.value('lightgray'), legend=None)
).transform_filter(click).properties(width = 600, height=300)

alt.hconcat(bars, points)

alt.HConcatChart(...)

## 2.3 Two-way Filtering

In [101]:
alt.Tooltip?

Init signature:
alt.Tooltip(
    shorthand=Undefined,
    aggregate=Undefined,
    bandPosition=Undefined,
    bin=Undefined,
    condition=Undefined,
    field=Undefined,
    format=Undefined,
    formatType=Undefined,
    timeUnit=Undefined,
    title=Undefined,
    type=Undefined,
    **kwds,
)
Docstring:     
Tooltip schema wrapper

Mapping(required=[shorthand])

Parameters
----------

shorthand : string
    shorthand for field, aggregate, and type
aggregate : :class:`Aggregate`
    Aggregation function for the field (e.g., ``"mean"``, ``"sum"``, ``"median"``,
    ``"min"``, ``"max"``, ``"count"`` ).

    **Default value:** ``undefined`` (None)

    **See also:** `aggregate <https://vega.github.io/vega-lite/docs/aggregate.html>`__
    documentation.
bandPosition : float
    Relative position on a band of a stacked, binned, time unit, or band scale. For
    example, the marks will be positioned at the beginning of the band if set to ``0``,
    and at the middle of the band if set to

In [105]:
alt.MarkConfig?

Init signature:
alt.MarkConfig(
    align=Undefined,
    angle=Undefined,
    aria=Undefined,
    ariaRole=Undefined,
    ariaRoleDescription=Undefined,
    aspect=Undefined,
    baseline=Undefined,
    blend=Undefined,
    color=Undefined,
    cornerRadius=Undefined,
    cornerRadiusBottomLeft=Undefined,
    cornerRadiusBottomRight=Undefined,
    cornerRadiusTopLeft=Undefined,
    cornerRadiusTopRight=Undefined,
    cursor=Undefined,
    description=Undefined,
    dir=Undefined,
    dx=Undefined,
    dy=Undefined,
    ellipsis=Undefined,
    endAngle=Undefined,
    fill=Undefined,
    fillOpacity=Undefined,
    filled=Undefined,
    font=Undefined,
    fontSize=Undefined,
    fontStyle=Undefined,
    fontWeight=Undefined,
    height=Undefined,
    href=Undefined,
    innerRadius=Undefined,
    interpolate=Undefined,
    invalid=Undefined,
    limit=Undefined,
    lineBreak=Undefined,
    lineHeight=Undefined,
    opacity=Undefined,
    order=Undefined,
    orient=Undefined,
    outerR

In [107]:
brush = alt.selection_interval(empty=True)

plot1 = alt.Chart(data).mark_point(tooltip={"content":'data'}).encode(
    x='gdp_capita:Q',
    y = 'life_expectancy:Q',
    color=alt.condition(brush, alt.value('red'), alt.value('lightgray'))
).add_params(brush).properties(width=400)

plot2 = alt.Chart(data).mark_point(tooltip={"content":'data'}).encode(
    x='gdp_capita:Q',
    y = 'fertility:Q',
    color=alt.condition(brush, alt.value('red'), alt.value('lightgray')),
    tooltip=['name','gdp_capita','fertility']
).add_params(brush).properties(width=400)

plot1 | plot2

alt.HConcatChart(...)

In [110]:
alt.Chart(data).mark_circle().encode(
    x = 'gdp_capita',
    y = alt.Y('life_expectancy').scale(zero=False),
    color = 'fertility',
    size = 'fertility'
).properties(width='container')

alt.Chart(...)

In [114]:
# Until now: How does GDP per Capita relate to Life Expectancy (2020)
# Next : Create a slider to select the year

alt.Chart(df[df.year==1990]).mark_bar().encode(
    x = 'region',
    y = 'mean(gdp_capita)'
).properties(width='container')

alt.Chart(...)

# Further Resources

## Altair

- [Population Pyramid Over Time](https://altair-viz.github.io/gallery/us_population_pyramid_over_time.html)
- [Multiline Highlight](https://altair-viz.github.io/gallery/multiline_highlight.html)
- [Multiline Tooltip](https://altair-viz.github.io/gallery/multiline_tooltip.html)
- [Multiple Interactions](https://altair-viz.github.io/gallery/multiple_interactions.html)
- [Selection Detail](https://altair-viz.github.io/gallery/select_detail.html)

## Inspiration: Eye-Catching Interactive Charts

- http://mvp.columnfivemedia.com/
- https://informationisbeautiful.net/visualizations/gender-pay-gap/

